[py041 Питон сетевой обмен по UDP протоколу](#py041-Питон-сетевой-обмен-по-UDP-протоколу)   
[py042 Передача файлов по сети](#py042-Передача-файлов-по-сети)   
[py043 Питон программа управления шаговым двигателем](#py043-Питон-программа-управления-шаговым-двигателем)   
[py044 Питон многопоточные клиент и сервер UDP](#py044-Питон-многопоточные-клиент-и-сервер-UDP)   
[py045 Питон система ведущий-ведомые в локальной сети](#py045-Питон-система-ведущий-ведомые-в-локальной-сети)   
[py046 Питон парсинг HTML страницы](#py046-Питон-парсинг-HTML-страницы)   
[py047 Питон работа по Modbus RTU](#py047-Питон-работа-по-Modbus-RTU)   
[py048 Консольное приложение для работы по протоколу Modbus RTU](#py048-Консольное-приложение-для-работы-по-протоколу-Modbus-RTU)   
[py049 Питон управляем Экселем](#py049-Питон-управляем-Экселем)   


#### py040 Питон - сырое чтение дисков
#### py041 Питон сетевой обмен по UDP протоколу
#### py042 Передача файлов по сети
#### py043 Питон программа управления шаговым двигателем
#### py044 Питон многопоточные клиент и сервер UDP
#### py045 Питон система ведущий-ведомые в локальной сети
#### py046 Питон парсинг HTML страницы
#### py047 Питон работа по Modbus RTU
#### py048 Консольное приложение для работы по протоколу Modbus RTU
#### py049 Питон управляем Экселем


#### py040 Питон - сырое чтение дисков
#### py041 Питон сетевой обмен по UDP протоколу
#### py042 Передача файлов по сети
#### py043 Питон программа управления шаговым двигателем
#### py044 Питон многопоточные клиент и сервер UDP
#### py045 Питон система ведущий-ведомые в локальной сети
#### py046 Питон парсинг HTML страницы
#### py047 Питон работа по Modbus RTU
#### py048 Консольное приложение для работы по протоколу Modbus RTU
#### py049 Питон управляем Экселем
